In [1]:
import os
import pandas as pd
import librosa
import glob
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam,RMSprop
from keras.utils import np_utils
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report 

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def parser(row):
   # function to load files and extract features
   file_name = row.FILE
   print(file_name)
   # handle exception to check if there isn't a file which is corrupted
   try:
      # here kaiser_fast is a technique used for faster extraction
      X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
      # we extract mfcc feature from data
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
#       XD = librosa.stft(X)
#       Xdb = librosa.amplitude_to_db(abs(XD))
   except Exception as e:
      print(e)
      print("Error encountered while parsing file: ", file_name)
      return None, None
   
#    feature =np.hstack((mfccs,Xdb)) 
   feature=mfccs
   label = row.NAME
   #print([feature, label])
   return  pd.Series([feature, label])

In [ ]:

data_dir = 'C:\\Users\\Arpit\\Desktop\\SDP\\Dataset\\LibriSpeech'
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
temp = train.apply(parser, axis=1)
temp.columns = ['feature', 'label']
#print(temp)


In [ ]:

valdation = pd.read_csv(os.path.join(data_dir, 'validation.csv'))
temp1 = train.apply(parser, axis=1)
temp1.columns = ['feature', 'label']
#print(temp1)

In [ ]:
X = np.array(temp.feature.tolist())
spk_label = np.array(temp.label.tolist())
print(spk_label)
lb = LabelEncoder()
import sys
np.set_printoptions(threshold=sys.maxsize)
y = np_utils.to_categorical(lb.fit_transform(spk_label))

print(y)

In [ ]:
val_x = np.array(temp1.feature.tolist())
val_y = np.array(temp1.label.tolist())

lb = LabelEncoder()

val_y = np_utils.to_categorical(lb.fit_transform(val_y))



In [ ]:
# print("#######################learning model started######################")
# num_labels = y.shape[1]
# filter_size = 2

#  build model
# model = Sequential()

# model.add(Dense(40, input_shape=(40,)))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

# model.add(Dense(25))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

# model.add(Dense(10))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

# model.add(Dense(num_labels))
# model.add(Activation('softmax'))

# model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# print("#######################learning model compile over ######################")

In [ ]:
print("#######################learning model started######################")
num_labels = y.shape[1]
filter_size = 2

# build model
model = Sequential()

model.add(Dense(40, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(25))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=RMSprop())

# model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
print("#######################learning model compile over ######################")

In [ ]:
print("#######################learning model training######################")
# model.fit(X, y, batch_size=32, epochs=10000, validation_data=(val_x, val_y))

history = model.fit(X, y, batch_size=32, epochs=1000, verbose=2, validation_data=(val_x, val_y))
score = model.evaluate(	val_x,val_y,verbose=0)

print('Test Loss:',score[0])
print('Test accuracy:',score[1])


In [ ]:
model.save(os.path.join('.', 'test1.model'))

In [ ]:
test = pd.read_csv(os.path.join('.', 'test.csv'))
temp = test.apply(parser, axis=1)
temp.columns = ['feature', 'label']


X = np.array(temp.feature.tolist())
y = np.array(temp.label.tolist())

lb = LabelEncoder()

y = np_utils.to_categorical(lb.fit_transform(y))




In [ ]:
# print(X[0])
predictions = model.predict(X)


In [ ]:
predictions = model.predict(X)

print(predictions)
print(y)
# Evaluate predictions
print(accuracy_score(y, predictions.round()))
rounded_y=np.argmax(y, axis=1)
print(confusion_matrix(rounded_y, predictions.argmax(axis=1)))
print(classification_report(rounded_y, predictions.argmax(axis=1)))

In [ ]:
print(predictions)